In [ ]:
print('Started...')
import glob
import pandas as pd
import pydicom
import os
import matplotlib.pyplot as plt
import png
import time
import numpy as np
import os
from fnmatch import fnmatch


# Gobals
cS = 256
DICOM_FILES = '/vol/vssp/cvpwrkspc01/scratch/wm0015/dcm/malignant/1.2.840.113681.2230565232.95*'
#DICOM_FILES = '/vol/vssp/cvpwrkspc01/scratch/wm0015/dcm/normal/1.2.840.113681.2230565232.95*'
DICOM_FILES = '/vol/vssp/cvpwrkspc01/scratch/wm0015/download/IMAGE_DATABASE/PUBLIC_SHARE/IMAGES/STANDARD_SET/1/optm10'
#DICOM_FILES = '/vol/vssp/cvpwrkspc01/scratch/wm0015/download/IMAGE_DATABASE/PUBLIC_SHARE/IMAGES/STANDARD_SET/1'


# File walk method
def getFiles():
    fMal = []
    for path, subdirs, files in os.walk(DICOM_FILES):
        for name in files:
            if fnmatch(name, '*.dcm'):
                fMal.append(os.path.join(path, name))

    print(len(fMal), ' dicom images found')


    # Load dicom files into np array
    malignant = np.array([])
    dicomImg = np.array([])
    count = 0
    print(len(fMal), ' Malignant found')
    print('Loading images...')
    for f in fMal:
        dicomImg = np.append(dicomImg, pydicom.dcmread(f))
        count += 1
        print(count, '/', len(fMal))
    return dicomImg, fMal



    print('name:\n', dicomImg[0].PresentationIntentType)

def deletePreProcessed(dicomImg, fMal):
    forPresentationCount = 0
    toDelete = []
    for index, _ in enumerate(dicomImg):
        tmp = _.PresentationIntentType
        if tmp == 'FOR PRESENTATION':
            forPresentationCount += 1
        else:
            toDelete.append(index)
    print('dicomImg len:', len(dicomImg))
    np.delete(dicomImg, toDelete)
    mask = np.ones(len(dicomImg), dtype=bool)
    mask[toDelete] = False
    dicomImg = dicomImg[mask]
    fMal = np.asarray(fMal)
    fMal = fMal[mask]
    print('\nTotal DICOM: ', len(dicomImg))
    print('Total FOR PROCESSING: ', forPresentationCount)
    return dicomImg, fMal

   
# Import xls file, extract ROI coords, get pixel array from DICOM image
def buildDict(dicomImg, fMal):
    xls = pd.ExcelFile('/vol/vssp/cvpwrkspc01/scratch/wm0015/download/batch_1_IMAGE.xls')
    sheet = xls.parse(1)

    # Create a dict where the key is the image name
    # Each key has the image, and coords
    img = {}
    for i in range(len(fMal)):
        key = os.path.basename(fMal[i])[:-4]
        # Find row in the xml file that holds the img info
        try:
            indx = [_==key for _ in sheet['ImageSOPIUID']].index(True) # ImageSOPIUID, ReferencedSOPInstanceUID
            img.update({key:{}})
            img[key].update({'img': dicomImg[i].pixel_array})
            img[key].update({'x': [sheet['X1'][indx], sheet['X2'][indx]]})
            img[key].update({'y': [sheet['Y1'][indx], sheet['Y2'][indx]]})
        except ValueError:
            print('MY_ERROR: key not found:\n', key)

    print(len(img), 'DICOM images extracted')
    # print(img)
    # Crop the images to given ROI
    toDelete = [] # Keep track of error causing keys and delete after loop
    for key in img:
        try:
            tmp = img[key]['img']
            x = img[key]['x']
            y = img[key]['y']
            x = [int(x[0]), int(x[1])]
            y = [int(y[0]), int(y[1])]
            #x is width, y is height
            #in numpy array, y,x
            img[key].update({'cropROI':tmp[y[0]:y[1], x[0]:x[1]]}) 
        except:
            print('ROI extraction failed...Removing key\nkey:  ',key, '\nx  :', x,'\ny:  ',y)
            toDelete.append(key)

    #img_calc = img['1.2.840.113681.2230565232.954.3504500766.32'] 
    for _ in toDelete:
        img.pop(_)
    return img


# Write images to disk with markers and basic crop    
def writeMarkedImages(img): 
    for key in img:
        plt.figure(figsize=(20,20))
        marker = [(img[key]['x'][1] + img[key]['x'][0])/2, (img[key]['y'][1] + img[key]['y'][0])/2 ]   
        plt.imshow(img[key]['img']/16383, cmap='gray', vmin=0, vmax=0.2)
        plt.plot(marker[0], marker[1], marker='x', color=[1,0,1], markersize=30)
        plt.savefig('/vol/vssp/cvpwrkspc01/scratch/wm0015/markers/tmp/' + key +'_full.png')
        #plt.show()
        plt.close()

        plt.figure(figsize=(20,20))
        plt.imshow(img[key]['cropROI'], cmap='gray') 
        plt.savefig('/vol/vssp/cvpwrkspc01/scratch/wm0015/markers/tmp/' + key +'_crop.png')   
        #plt.show()
        plt.close()

# Crop the images so that the ROI is centred but all crops are the same size
def computeCrops(img):
    for key in img:
        tmp = img[key]['img']
        x = img[key]['x']
        y = img[key]['y']
        c = [round((x[0]+x[1])/2), round((y[0]+y[1])/2)]
        # Pad images before cropping (wrap around)
        pad = 1000
        tmp = np.pad(tmp, pad, mode='wrap')
        img[key].update({'crop': tmp[int(c[1]-cS/2+pad):int(c[1]+cS/2+pad), int(c[0]-cS/2+pad):int(c[0]+cS/2+pad)]})
        # Reshape from (256, 256) to (256, 256, 1)
        img[key]['crop'] = np.reshape(img[key]['crop'],(img[key]['crop'].shape[0], img[key]['crop'].shape[1], 1))
    return img
    
# Find bit depth
def findBitDepth(img):
    maxmax = 0
    for key in img:
        tmp = img[key]['img']
        print(np.amax(tmp))
        if np.amax(tmp) > maxmax:
            maxmax = np.amax(tmp)
    print('The largest value is: ', maxmax)
    
# Find average ROI size
def findAverageROISize(img):
    totalX = 0
    totalY = 0
    for key in img:
        x = img[key]['x']
        y = img[key]['y']
        totalX += x[1] - x[0]
        totalY += y[1] - y[0]
    print('Average ROI width: ', totalX/len(img), '\nAverage ROI length: ', totalY/len(img))
    
# View crops / save to disk
def writeCropsToDisk(img):
    count = 0
    for key in img:
        count+=1
        f = open('/vol/vssp/cvpwrkspc01/scratch/wm0015/batch1_crop/' + key + '.png', 'wb')
        w = png.Writer(width = cS, height = cS, bitdepth=16, greyscale=True)
        w.write(f, img[key]['crop'])
        f.close()
        print(count, '/', len(img))
        if count == -1:
            break
            
def buildArrayForPickle(img):
    allCrops = []
    for key in img:
        allCrops.append(img[key]['crop'])
    return allCrops
        
def savePickle(ob):
    print('Pickling...')
    with open('/vol/vssp/cvpwrkspc01/scratch/wm0015/batch1_crop256.pickle', 'wb') as output:
        pickle.dump(ob, output, pickle.HIGHEST_PROTOCOL)
        
#def getNormals


def main():
    dicomImg, fileList = getFiles()
    dicomImg, fileList = deletePreProcessed(dicomImg, fileList)
    img = buildDict(dicomImg, fileList)
    img = computeCrops(img)
    writeMarkedImages(img)
    findBitDepth(img)
    writeCropsToDisk(img)
    savePickle(buildArrayForPickle(img)) 
    findAverageROISize(img)


if __name__ == "__main__":
    main()

print('Done')

Started...
10  dicom images found
10  Malignant found
Loading images...
1 / 10
2 / 10
3 / 10
4 / 10
5 / 10
6 / 10
7 / 10
8 / 10
9 / 10
10 / 10
dicomImg len: 10

Total DICOM:  5
Total FOR PROCESSING:  5
MY_ERROR: key not found:
 1.2.826.0.1.3680043.9.3218.1.1.68298180.9661.1516806470059.271.0
MY_ERROR: key not found:
 1.2.826.0.1.3680043.9.3218.1.1.68298180.9661.1516806470059.287.0
MY_ERROR: key not found:
 1.2.826.0.1.3680043.9.3218.1.1.68298180.9661.1516806470059.279.0
2 DICOM images extracted


In [15]:
print('hello world')
import pickle

hello world


In [28]:

def test():
    print(x)
    x = x+1
    
x = 2
test()


UnboundLocalError: local variable 'x' referenced before assignment